In [6]:
# Imports
import os
import getpass

import numpy

import nipype
import nibabel
import jinja2

import nipype.interfaces.io as nio


from nipype.interfaces.base import BaseInterface
from nipype.interfaces.utility import IdentityInterface

from nipype.interfaces.base import BaseInterfaceInputSpec, File, TraitedSpec, traits
from nipype import Node, Function, Workflow
from nipype.interfaces.io import BIDSDataGrabber
from nilearn.plotting import plot_roi
from nilearn import image
from bids.layout import BIDSLayout
from nipype.pipeline import engine as pe

from matplotlib import pyplot as plt
from os.path import join
from bids.tests import get_test_data_path

210312-17:02:43,887 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
user = getpass.getuser()

if user == 'ari':
    bids_ds_path = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_andres/'

if user == 'andres':
    bids_ds_path = '/home/andres/Desktop/BIDS/DS001_BIDS_andres/'

deriv_cmp_dir = join(bids_ds_path, "derivatives", "cmp")
deriv_nipype_dir = join(bids_ds_path, "nipype", "sub-01", "ses-01")

In [3]:
class OverlayT1wAtlasInputSpec(BaseInterfaceInputSpec):
    
    atlas_file = traits.List(exists=True, mandatory=True, desc='atlas file') # Do not set exists=True !!
    t1w_file = traits.List(exists=True, mandatory=True, desc='t1w file')

    
class OverlayT1wAtlasOutputSpec(TraitedSpec):
    fig_file = File(desc='png overlay')
    
    

class OverlayT1wAtlas(BaseInterface):
    input_spec = OverlayT1wAtlasInputSpec
    output_spec = OverlayT1wAtlasOutputSpec
    
    def _run_interface(self, runtime):

        # Call our python code here:
        self.fig_file = self._overlay_t1w_parcellation(
            self.inputs.t1w_file,
            self.inputs.atlas_file
        )

        # And we are done
        return runtime
    
    def _overlay_t1w_parcellation(self, t1w_path, atlas_path):
        

        t1w_image = nibabel.load(t1w_path[0])

        parcellation = 'scale5'
                
        roi_path_selected = [elem for elem in atlas_path if parcellation in elem]
        roi_image = nibabel.load(roi_path_selected[0])

        output_file = bids_ds_path + 'derivatives/nipype/sub-01/overlay.png'
        fig_file = plot_roi(roi_image, t1w_image, output_file = output_file)
        
        return output_file
    
    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['fig_file'] = self.fig_file
        return outputs

In [4]:
class ReportInputSpec(BaseInterfaceInputSpec):
    fig_file = File(desc='png overlay')

    
class ReportOutputSpec(TraitedSpec):
    report_file = File(desc='HTML report file')
    

class Report(BaseInterface):
    input_spec = ReportInputSpec
    #output_spec = ReportOutputSpec
    
    def _run_interface(self, runtime):
        
        # Call our python code here:
        self.fig_file = self._generate_html(
            self.inputs.fig_file
        )
        
        # And we are done
        return runtime
    
    def _generate_html(self, fig_file):
        print("Llego!")
        title = 'Hi this is my title'
        outputfile = '/home/andres/Desktop/BIDS/automatedHTML/myfile.html'
        subs = jinja2.Environment(
                      loader=jinja2.FileSystemLoader('/home/andres/Desktop/BIDS/automatedHTML/templates')
                      ).get_template('template.html').render(title=title,figure=fig_file)
        # lets write the substitution to a file
        with open(outputfile,'w') as f: f.write(subs)

In [5]:
bg = Node(BIDSDataGrabber(index_derivatives=True), name='bids-grabber')
bg.inputs.base_dir = bids_ds_path

bg.inputs.subject = '01'

bg.inputs.output_query = {'atlas': {'scope': 'Connectome Mapper',
                                  'suffix': 'atlas',
                                  'extensions': ['nii.gz']},
                         't1w': {'scope': 'Connectome Mapper',
                                  'suffix': 'T1w',
                                  'extensions': ['nii.gz']},
                         }

overlay = Node(OverlayT1wAtlas(), name='overlay_fig')
report = Node(Report(), name='report')
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory = deriv_nipype_dir

In [6]:
workflow = Workflow("overlay_workflow")

workflow.connect([(bg, overlay,
                   [('atlas', 'atlas_file'),
                    ('t1w','t1w_file')]
                  )])
workflow.connect(overlay, 'fig_file', datasink, 'fig_input')
workflow.connect(overlay, 'fig_file', report, 'fig_file')

workflow.run()

210312-17:02:45,205 nipype.workflow INFO:
	 Workflow overlay_workflow settings: ['check', 'execution', 'logging', 'monitoring']
210312-17:02:45,212 nipype.workflow INFO:
	 Running serially.
210312-17:02:45,213 nipype.workflow INFO:
	 [Node] Setting-up "overlay_workflow.bids-grabber" in "/tmp/tmpkkgxve8h/overlay_workflow/bids-grabber".
210312-17:02:45,217 nipype.workflow INFO:
	 [Node] Running "bids-grabber" ("nipype.interfaces.io.BIDSDataGrabber")
210312-17:02:45,932 nipype.workflow INFO:
	 [Node] Finished "overlay_workflow.bids-grabber".
210312-17:02:45,933 nipype.workflow INFO:
	 [Node] Setting-up "overlay_workflow.overlay_fig" in "/tmp/tmpwhp_d_59/overlay_workflow/overlay_fig".
210312-17:02:45,937 nipype.workflow INFO:
	 [Node] Running "overlay_fig" ("__main__.OverlayT1wAtlas")
210312-17:02:48,25 nipype.workflow INFO:
	 [Node] Finished "overlay_workflow.overlay_fig".
210312-17:02:48,27 nipype.workflow INFO:
	 [Node] Setting-up "overlay_workflow.report" in "/tmp/tmpdrp8z346/overlay_w